In [1]:
import csv
import numpy as np
from sklearn.cross_validation import KFold
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import LinearSVC

C:\Users\ar1\Anaconda3\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [19]:
def findCommunities():
    communities = []
    with open('C:\\Users\\ar1\\Documents\ML\\Project\\Crime Prediction Data(1)\\Crime Prediction Data\\communities-crime-clean.csv', 'r') as csvfile:
        filereader = csv.reader(csvfile)
        j = 0
        for i in filereader:
            if j > 0:
                if i[1] not in communities:
                    communities.append(i[1])
            j+=1
    return communities


In [20]:
def extractData(filename,communities,addstate = False,addcommunity = False):
    features = []
    x=[]
    y=[]
    with open(filename, 'r') as csvfile:
        filereader = csv.reader(csvfile)
        j = 0
        for i in filereader:
            if j > 0:
                data = i[3:len(i)-1]
                if addstate:
                    state = [0.0]*56
                    state[int(i[0])-1] = 1.0
                    data = data + state
                    features = features + ["state"+str(i) for i in range(56)]
                if addcommunity:
                    tmp = [0.0]*len(communities)
                    tmp[communities.index(i[1])] = 1.0
                    data = data + tmp
                    features = features + communities
                vect = np.array(data)
                #vect = np.array(i[3:len(i)-1])
                x.append(vect.astype(np.float))
                if float(i[-1]) > 0.1:
                    y.append(1)
                else:
                    y.append(0)
            else:
                features = i[3:len(i)-1]
            j+=1
        y = np.array(y)
        x = np.array(x)
        features = np.array(features)
        return x,y,features


In [21]:
def calcualtePercentage(labels):
    n = len(labels)
    sumTrue = sum(labels)*1.0
    sumFalse = (n - sumTrue)*1.0
    return sumTrue/n,sumFalse/n

In [22]:
def gaussianNB():
    return GaussianNB()

In [23]:
def confusionMatrix(true,predicted,labels = [1,0]):
    return confusion_matrix(true,predicted,labels)


In [24]:
def find_confusion_matrix(actual,predicted,clabels):
    cm= []
    for i in clabels:
        tmp =[0]*len(clabels)
        
        for j in range(len(actual)):
            if actual[j]== i and actual[j] == predicted[j]:
                tmp[clabels.index(i)] += 1
            elif actual[j]== i and actual[j] != predicted[j]:
                tmp[clabels.index(predicted[j])] += 1
        cm.append(tmp)
    return np.array(cm)

In [25]:
def find_accuracy(matrix):
    return np.trace(matrix)*1.0/np.sum(matrix)

In [26]:
def find_precision(matrix):
    pres = []
    x = np.sum(matrix,axis=0)
    for i in range(len(matrix)):
        for j in range(len(matrix)):
            if i == j:
                pres.append(matrix[i][j]*1.0/x[i])
    return pres

In [27]:
def find_recall(matrix):
    rec = []
    x = np.sum(matrix,axis=1)
    for i in range(len(matrix)):
        for j in range(len(matrix)):
            if i == j:
                rec.append(matrix[i][j]*1.0/x[i])
                return rec

In [28]:
def find_fmeasure(prec,rec):
    tmp = []
    for i,j in zip(prec,rec):
        tmp.append(2.0*(i*j)/(i+j))
    return tmp

In [29]:
def evaluation(acutal,predicted,clabels=[1,0]):
    confmatrix = find_confusion_matrix(acutal,predicted,clabels)
    print ("Confusion Matrix")
    print (confmatrix)
    accuracy = find_accuracy(confmatrix)
    print ("Accuracy", accuracy)
    precision = find_precision(confmatrix)
    print ("Precision", precision)
    recall = find_recall(confmatrix)
    print ("Recall", recall)
    f_score =find_fmeasure(precision,recall)
    print ("F_score", f_score)

In [30]:
def do_cross_validation(X, y,clf, n_folds=5):
    cv = KFold(len(y), n_folds)
    accuracies = []
    for train_ind, test_ind in cv: 
        clf.fit(X[train_ind], y[train_ind])
        predictions = clf.predict(X[test_ind])
        evaluation(y[test_ind],predictions)
        accuracies.append(accuracy_score(y[test_ind], predictions))
    avg = np.mean(accuracies)
    return avg

In [31]:
distintCommunities = findCommunities()

In [34]:
filename= 'C:\\Users\\ar1\\Documents\ML\\Project\\Crime Prediction Data(1)\\Crime Prediction Data\\communities-crime-clean.csv'
X,Y,Features = extractData(filename, distintCommunities)

In [35]:
calcualtePercentage(Y)

(0.62719518314099343, 0.37280481685900652)

In [36]:
gnb= gaussianNB()
gnb.fit(X[:1000],Y[:1000])
predictedY=gnb.predict(X[1000:])

In [37]:
confusionMatrix(Y[1000:],predictedY)

array([[376, 165],
       [ 49, 403]])

In [38]:
evaluation(Y[1000:],predictedY)

Confusion Matrix
[[376 165]
 [ 49 403]]
Accuracy 0.784491440081
Precision [0.88470588235294123, 0.70950704225352113]
Recall [0.69500924214417747]
F_score [0.77846790890269169]


In [39]:
do_cross_validation(X[:1000],Y[:1000],gnb,10)


Confusion Matrix
[[59 15]
 [ 9 17]]
Accuracy 0.76
Precision [0.86764705882352944, 0.53125]
Recall [0.79729729729729726]
F_score [0.83098591549295775]
Confusion Matrix
[[73 18]
 [ 2  7]]
Accuracy 0.8
Precision [0.97333333333333338, 0.28000000000000003]
Recall [0.80219780219780223]
F_score [0.87951807228915668]
Confusion Matrix
[[65 25]
 [ 0 10]]
Accuracy 0.75
Precision [1.0, 0.2857142857142857]
Recall [0.72222222222222221]
F_score [0.83870967741935476]
Confusion Matrix
[[62 20]
 [ 6 12]]
Accuracy 0.74
Precision [0.91176470588235292, 0.375]
Recall [0.75609756097560976]
F_score [0.82666666666666655]
Confusion Matrix
[[41 11]
 [ 0 48]]
Accuracy 0.89
Precision [1.0, 0.81355932203389836]
Recall [0.78846153846153844]
F_score [0.88172043010752688]
Confusion Matrix
[[62 25]
 [ 1 12]]
Accuracy 0.74
Precision [0.98412698412698407, 0.32432432432432434]
Recall [0.71264367816091956]
F_score [0.82666666666666677]
Confusion Matrix
[[45 19]
 [ 5 31]]
Accuracy 0.76
Precision [0.90000000000000002, 0.62]


0.74299999999999988

Explaination: Cross Validation
2-a-i): The accuracy,precision and recall can be found above. 

2-a-iii) When comparing the  above cross validation results(74.2%) with decision tree's cross validation results (76 % accuracy), decision tree's cross validation results turned out to be better. 

In [40]:
accuracy_score(Y[1000:],predictedY)

0.78449144008056393

In [41]:
sumCorrect =0
for i in range(1000,len(Y)):
    if Y[i]==predictedY[i-1000]:
        sumCorrect += 1
sumCorrect*1.0/len(Y[1000:]) 

0.7844914400805639